<a href="https://colab.research.google.com/github/pbannuru/NLP-ALL_STUFF/blob/main/TEXT_GENERATION_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import time


In [ ]:
os.makedirs("data",exist_ok=True)

In [ ]:
# !curl https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt >"./data/shakespeare.txt"

In [ ]:
!curl https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt >"./pandas.pdf"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  46.2M      0 --:--:-- --:--:-- --:--:-- 46.2M


In [ ]:
class Config:
    path_to_file = os.path.join("data","shakespeare.txt")
    seq_length = 100  # max seq to process
    
    Batch_size =64   # 
    Buffer_size = 10000 # keep data in ram , while training randomly pick data from (buffer size) storage
                                
    embedding_dim = 256                            
    rnn_units = 1024    # we are using 1024 GRU here, GRU is a RNN technique                    
    EPOCHS = 30                            
    checkpoint_dir = "./training_ckpt"         # storing temp model                    
                                
     

In [ ]:
text = open(Config.path_to_file,'rb').read()
text[:100]

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [ ]:
text = open(Config.path_to_file,'rb').read().decode(encoding='utf-8')
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [ ]:
with open(Config.path_to_file) as f:
    t=f.read()

In [ ]:
t[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [ ]:
set(text)

{'\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [ ]:
vocab = sorted(set(text))
len(vocab)

65

In [ ]:
# vectorise the text

char2idx = {uniChar:idx for idx,uniChar in enumerate(vocab)}
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [ ]:
text_as_int = np.array([char2idx[c] for c in text])
text_as_int

array([18, 47, 56, ..., 45,  8,  0])

In [ ]:
for c in text[:50] :
    print (c,end='')
    

First Citizen:
Before we proceed any further, hear

In [ ]:
for c in text[:50] :
    print(char2idx[c],',',end='')

18 ,47 ,56 ,57 ,58 ,1 ,15 ,47 ,58 ,47 ,64 ,43 ,52 ,10 ,0 ,14 ,43 ,44 ,53 ,56 ,43 ,1 ,61 ,43 ,1 ,54 ,56 ,53 ,41 ,43 ,43 ,42 ,1 ,39 ,52 ,63 ,1 ,44 ,59 ,56 ,58 ,46 ,43 ,56 ,6 ,1 ,46 ,43 ,39 ,56 ,

In [ ]:
text[:13],text_as_int[:13]

('First Citizen', array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]))

In [ ]:
examples_per_epoch = len(text)//(Config.seq_length +1)   # max length we are going to process
examples_per_epoch                                    # +1 here is Ex: hello = 4+1 hell is input and ouput = 0

11043

In [ ]:
idx2char = {val:key for key,val in char2idx.items()}


In [ ]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)   # creatig training examples

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [ ]:
sequences = char_dataset.batch(Config.seq_length+1,drop_remainder=True)

for item in sequences.take(5):
    print(item.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49]
[52 53 61  1 15 39 47 59 57  1 25 39 56 41 47 59 57  1 47 57  1 41 46 47
 43 44  1 43 52 43 51 63  1 58 53  1 58 46 43  1 54 43 53 54 50 43  8  0
  0 13 50 50 10  0 35 43  1 49 52 53 61  5 58  6  1 61 43  1 49 52 53 61
  5 58  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 24 43 58  1
 59 57  1 49 47]
[50 50  1 46 47 51  6  1 39 52 42  1 61 43  5 50 50  1 46 39 60 43  1 41


In [ ]:
for item in sequences.take(5):
    print("".join([idx2char[c] for c in item.numpy()]))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki
ll him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be d
one: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citi


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text,target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example,target_example in dataset.take(1):
    print("input_data:\n")
    print("".join([idx2char[i] for i in input_example.numpy()]))
    print("target_data:\n")
   

input_data:

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
target_data:



In [ ]:
# creating training batches

dataset = dataset.shuffle(Config.Buffer_size).batch(Config.Batch_size,drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
Config.vocab_size = len(vocab)

In [ ]:
def build_model(vocab_size,embedding_dim,rnn_units,batch_size):
    model= tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size,embedding_dim,batch_input_shape=[batch_size,None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,    #no output is given by each GRU
                            stateful=True,            # for one epoch to next one hidden state =0,means False,for next epoch it starts with zero so it take chunk of data and predict but if it is TRUE it memorise whole para and gives result.
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    
    
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=Config.vocab_size,
    embedding_dim=Config.embedding_dim,
    rnn_units=Config.rnn_units,
    batch_size = Config.Batch_size
)

In [ ]:
def loss(Labels,Logits):
    return tf.keras.losses.sparse_categorical_crossentropy(Labels,Logits,from_logits=True)

In [ ]:
model.compile(optimizer='adam',loss=loss)

In [ ]:
checkpoint_prefix = os.path.join(Config.checkpoint_dir,"ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [ ]:
history = model.fit(dataset,epochs = Config.EPOCHS,callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 16s 52ms/step - loss: 2.6815
Epoch 2/30
172/172 [==============================] - 10s 52ms/step - loss: 1.9514
Epoch 3/30
172/172 [==============================] - 10s 52ms/step - loss: 1.6825
Epoch 4/30
172/172 [==============================] - 10s 53ms/step - loss: 1.5370
Epoch 5/30
172/172 [==============================] - 10s 53ms/step - loss: 1.4496
Epoch 6/30
172/172 [==============================] - 10s 54ms/step - loss: 1.3892
Epoch 7/30
172/172 [==============================] - 10s 54ms/step - loss: 1.3431
Epoch 8/30
172/172 [==============================] - 11s 55ms/step - loss: 1.3050
Epoch 9/30
172/172 [==============================] - 11s 55ms/step - loss: 1.2696
Epoch 10/30
172/172 [==============================] - 11s 55ms/step - loss: 1.2371
Epoch 11/30
172/172 [==============================] - 11s 56ms/step - loss: 1.2049
Epoch 12/30
172/172 [==============================] - 11s 56ms/step - loss: 1.1727
E

In [ ]:
(len(text)/Config.Batch_size)/(Config.seq_length+1)

172.55476485148515

In [ ]:
Config.seq_length

100

In [ ]:
# restoring checkpoint:

tf.train.latest_checkpoint(Config.checkpoint_dir)

'./training_ckpt/ckpt_30'

In [ ]:
model_from_ckpt = build_model(
    vocab_size=Config.vocab_size,
    embedding_dim=Config.embedding_dim,
    rnn_units=Config.rnn_units,
    batch_size = 1
)

In [ ]:
model_from_ckpt.load_weights(tf.train.latest_checkpoint(Config.checkpoint_dir))

model_from_ckpt.build(tf.TensorShape)

In [ ]:
model_from_ckpt.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_from_ckpt.summary() # batch size given 1

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#prediction
def generate_text(model,start_string,no_of_chars_to_gen =1000):
  input_val = [char2idx[s] for s in start_string] # text converted to int
  input_val = tf.expand_dims(input_val,0) # [] --> [1,]  1D to 2D

  text_generated = list()

  temperature = 1.0  # kind of normalisation to the prediction
  model.reset_states() # resetting the previous status of training
  for i in range(no_of_chars_to_gen):
    predictions = model(input_val)

    predictions=tf.squeeze(predictions,0)

    predictions=predictions / temperature

    #print(predictions)

    predicted_id= tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()

    input_val=tf.expand_dims([predicted_id],0)

    text_generated.append(idx2char[predicted_id])

  return start_string +"".join(text_generated)








In [ ]:
result = generate_text(model=model_from_ckpt,start_string="pavan kumar",no_of_chars_to_gen=1000)
print(result)

pavan kumark;
And, if we be, the shore of heaven:
I would be another for't; commanded there
And smooth the truth:
'Tis thoughts to see 'em!--
The mather will I learn the rootman bad as true?
Jointed all, that feed's incress and account,
Show thy descriet, and save yourself spoke
Fox our affaits of heads er passing path
Thy by the rosem prison.

CORIOLANUS:
Do, get thee to our tento, time
The flours of hell, and broke her thanks.

ABHORSON:
Lo, canst thou gave this blood there is no cause to stand, to Angelo were butcher'd, O, that I regreet
The patricians of you soundly kind
secution, and seventeties and danger that wine enemies
Of gracious Richard, England's king but Edward?
He was not yet behind in Padua. Dran, sir: sir!

AUTOLYCUS:
Away, away, begranato, there show'd me and
Self-whine and all, of late,
Which are heaven with bother's Chert is sich and heart
To have asher state and marriage, have accounted
I throne; the hearers will acquaint
My plain your sight as you had then to loun